In [ ]:
import mne
from sklearn.decomposition import FastICA, PCA
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib

In [ ]:
#Reading sqd file
input_fname = '/media/analogicalnexus/2568212B752CDB3B/MEG_Data/test_sess_data/R2501_resting_04.04.18.sqd'
stim_list =range(160,180)
raw=mne.io.read_raw_kit(input_fname, mrk=None, elp=None, hsp=None, stim=stim_list, slope='+', stimthresh=-1, preload=True,stim_code='channel')
raw.load_data()
# raw.plot()

In [ ]:
#Band pass filter
raw_40 = raw.filter(l_freq=0.1, h_freq=60.0)
# raw_40.plot()

In [ ]:
# # compute ICA
# ica = FastICA(n_components=193)
# S_ = ica.fit_transform(raw_40)  # Get the estimated sources
# A_ = ica.mixing_  # Get estimated mixing matrix

In [ ]:
events = mne.find_events(raw_40)
print(events)

In [ ]:
event_id = dict(r=155, n_r=156)  
tmin = -.2 
tmax = 1
baseline = (None,0) 
picks = mne.pick_types(raw_40.info, meg=True) 
 epochs = mne.Epochs(raw_40, events, event_id, tmin, tmax, proj=False, picks=picks, baseline=baseline) 

In [ ]:
evoked1 = epochs['tone1'].average()
evoked2 = epochs['tone2'].average()
evoked1.plot()
evoked2.plot()

In [ ]:
evoked1.plot(spatial_colors=True, gfp=True, ylim=dict(mag=[-300,300]))
evoked2.plot(spatial_colors=True, gfp=True, ylim=dict(mag=[-300,300]))
evoked1.plot_topomap(times=[.05, .10, .15],vmin=-300,vmax=300)
evoked2.plot_topomap(times=[.05, .10, .15],vmin=-300,vmax=300)
evoked_dict = dict() 
evoked_dict['low'] = evoked1
evoked_dict['high'] = evoked2
colors=dict(low="Crimson",high="CornFlowerBlue") 
mne.viz.plot_compare_evokeds(evoked_dict, colors=colors,
picks=picks, gfp=True)

In [ ]:

la=[0,1,2,3,39,41,42,43,44,52,58,67,71,80,82,83,84,85,108,130,131,132,133,134,135,136,151]
lp=[4,5,6,7,8,9,34,36,37,38,40,45,46,47,48,49,50,75,76,77,79,87,88,90,127,129,137]
ra=[20,22,23,24,26,59,60,61,62,63,65,89,92,95,99,100,114,115,116,117,118,145,147,148,152,155]
rp=[14,15,16,17,18,19,25,27,28,30,53,54,56,57,66,68,69,70,94,96,97,119,121,122,143,144]
lh=[0,1,2,3,39,41,42,43,44,52,58,67,71,80,82,83,84,85,108,130,131,132,133,134,135,136,151,4,5,6,7,8,9,34,36,37,38,40,45,46,47,48,49,50,75,76,77,79,87,88,90,127,129,137]
rh=[20,22,23,24,26,59,60,61,62,63,65,89,92,95,99,100,114,115,116,117,118,145,147,148,152,155, 14,15,16,17,18,19,25,27,28,30,53,54,56,57,66,68,69,70,94,96,97,119,121,122,143,144]
mne.viz.plot_compare_evokeds(evoked_dict, colors=colors,
picks=lh, gfp=True, ylim=dict(mag=[0,100]))